In [2]:
from google.cloud import storage
import json
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

bucket_name = "swebench_logs"
output_file = "output.json"

client = storage.Client()
bucket = client.bucket(bucket_name)

instance_status = {}

logging.info(f"Starting to process blobs in bucket: {bucket_name}")

json_reports = []

for blob in bucket.list_blobs():
    parts = blob.name.split('/')
    # We expect something like: instance_id/run_id/report.json
    # So we need at least 3 parts after splitting by '/'
    # Example: ["<instance_id>", "<run_id>", "report.json"]
    # if len(parts) < 3:
    #     logging.warning(f"Skipping blob {blob.name} as it does not match the expected structure")
    #     continue
    
    filename = parts[-1]
    if filename == 'report.json':
        instance_id = parts[0]
        run_id = parts[1]

        # Add the instance_id to the list of json_reports
        json_reports.append(instance_id)

        # Download and parse the JSON
        data = json.loads(blob.download_as_string())

        # The JSON should have a top-level key = instance_id, which has an object including 'resolved'
        instance_data = data.get(instance_id)
        if instance_data is None:
            logging.error(f"[{instance_id}]: {run_id} missing instance data in report.json")
            resolved = False
        else:
            resolved = instance_data.get("resolved")
            if resolved is None:
                logging.error(f"[{instance_id}]: {run_id} missing 'resolved' field in report.json")
                resolved = False
            else:
                resolved = bool(resolved)

        # If we haven't tracked this instance_id yet, initialize it
        if instance_id not in instance_status:
            instance_status[instance_id] = False

        # If this run for the instance is resolved, mark the instance as success
        if resolved == True:
            instance_status[instance_id] = True
        else:
            instance_status[instance_id] = False

logging.info("Finished processing all blobs.")

print(f"Found {len(json_reports)} json reports")

# Print instances count
print(f"Instances count: {len(instance_status)}")
# After processing all, write the instance_status map to a local JSON file
with open(output_file, 'w') as f:
    json.dump(instance_status, f, indent=2)

logging.info(f"Instance statuses written to {output_file}")

2024-12-11 14:47:03,042 - INFO - Starting to process blobs in bucket: swebench_logs
2024-12-11 14:48:02,975 - INFO - Finished processing all blobs.
2024-12-11 14:48:02,976 - INFO - Instance statuses written to output.json


Found 835 json reports
Instances count: 420
